In [15]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import torch.optim as optim
import torch.nn as nn
from PIL import Image
from tqdm import tqdm
from torchvision.models.segmentation import deeplabv3_resnet50, DeepLabV3_ResNet50_Weights
import torch.optim.lr_scheduler as lr_scheduler

In [16]:
base_dir = "CityscapesDataset"

In [17]:
# This block of code is taken from: https://github.molgen.mpg.de/mohomran/cityscapes/blob/master/scripts/helpers/labels.py#L55

from collections import namedtuple

#--------------------------------------------------------------------------------
# Definitions
#--------------------------------------------------------------------------------

# a label and all meta information
Label = namedtuple( 'Label' , [

    'name'        , # The identifier of this label, e.g. 'car', 'person', ... .
                    # We use them to uniquely name a class

    'id'          , # An integer ID that is associated with this label.
                    # The IDs are used to represent the label in ground truth images
                    # An ID of -1 means that this label does not have an ID and thus
                    # is ignored when creating ground truth images (e.g. license plate).

    'trainId'     , # An integer ID that overwrites the ID above, when creating ground truth
                    # images for training.
                    # For training, multiple labels might have the same ID. Then, these labels
                    # are mapped to the same class in the ground truth images. For the inverse
                    # mapping, we use the label that is defined first in the list below.
                    # For example, mapping all void-type classes to the same ID in training,
                    # might make sense for some approaches.

    'category'    , # The name of the category that this label belongs to

    'categoryId'  , # The ID of this category. Used to create ground truth images
                    # on category level.

    'hasInstances', # Whether this label distinguishes between single instances or not

    'ignoreInEval', # Whether pixels having this class as ground truth label are ignored
                    # during evaluations or not

    'color'       , # The color of this label
    ] )


#--------------------------------------------------------------------------------
# A list of all labels
#--------------------------------------------------------------------------------

# Please adapt the train IDs as appropriate for you approach.
# Note that you might want to ignore labels with ID 255 during training.
# Make sure to provide your results using the original IDs and not the training IDs.
# Note that many IDs are ignored in evaluation and thus you never need to predict these!

labels = [
    #       name                     id    trainId   category            catId     hasInstances   ignoreInEval   color
    Label(  'unlabeled'            ,  0 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'ego vehicle'          ,  1 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'rectification border' ,  2 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'out of roi'           ,  3 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'static'               ,  4 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'dynamic'              ,  5 ,      255 , 'void'            , 0       , False        , True         , (111, 74,  0) ),
    Label(  'ground'               ,  6 ,      255 , 'void'            , 0       , False        , True         , ( 81,  0, 81) ),
    Label(  'road'                 ,  7 ,        0 , 'ground'          , 1       , False        , False        , (128, 64,128) ),
    Label(  'sidewalk'             ,  8 ,        1 , 'ground'          , 1       , False        , False        , (244, 35,232) ),
    Label(  'parking'              ,  9 ,      255 , 'ground'          , 1       , False        , True         , (250,170,160) ),
    Label(  'rail track'           , 10 ,      255 , 'ground'          , 1       , False        , True         , (230,150,140) ),
    Label(  'building'             , 11 ,        2 , 'construction'    , 2       , False        , False        , ( 70, 70, 70) ),
    Label(  'wall'                 , 12 ,        3 , 'construction'    , 2       , False        , False        , (102,102,156) ),
    Label(  'fence'                , 13 ,        4 , 'construction'    , 2       , False        , False        , (190,153,153) ),
    Label(  'guard rail'           , 14 ,      255 , 'construction'    , 2       , False        , True         , (180,165,180) ),
    Label(  'bridge'               , 15 ,      255 , 'construction'    , 2       , False        , True         , (150,100,100) ),
    Label(  'tunnel'               , 16 ,      255 , 'construction'    , 2       , False        , True         , (150,120, 90) ),
    Label(  'pole'                 , 17 ,        5 , 'object'          , 3       , False        , False        , (153,153,153) ),
    Label(  'polegroup'            , 18 ,      255 , 'object'          , 3       , False        , True         , (153,153,153) ),
    Label(  'traffic light'        , 19 ,        6 , 'object'          , 3       , False        , False        , (250,170, 30) ),
    Label(  'traffic sign'         , 20 ,        7 , 'object'          , 3       , False        , False        , (220,220,  0) ),
    Label(  'vegetation'           , 21 ,        8 , 'nature'          , 4       , False        , False        , (107,142, 35) ),
    Label(  'terrain'              , 22 ,        9 , 'nature'          , 4       , False        , False        , (152,251,152) ),
    Label(  'sky'                  , 23 ,       10 , 'sky'             , 5       , False        , False        , ( 70,130,180) ),
    Label(  'person'               , 24 ,       11 , 'human'           , 6       , True         , False        , (220, 20, 60) ),
    Label(  'rider'                , 25 ,       12 , 'human'           , 6       , True         , False        , (255,  0,  0) ),
    Label(  'car'                  , 26 ,       13 , 'vehicle'         , 7       , True         , False        , (  0,  0,142) ),
    Label(  'truck'                , 27 ,       14 , 'vehicle'         , 7       , True         , False        , (  0,  0, 70) ),
    Label(  'bus'                  , 28 ,       15 , 'vehicle'         , 7       , True         , False        , (  0, 60,100) ),
    Label(  'caravan'              , 29 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0, 90) ),
    Label(  'trailer'              , 30 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0,110) ),
    Label(  'train'                , 31 ,       16 , 'vehicle'         , 7       , True         , False        , (  0, 80,100) ),
    Label(  'motorcycle'           , 32 ,       17 , 'vehicle'         , 7       , True         , False        , (  0,  0,230) ),
    Label(  'bicycle'              , 33 ,       18 , 'vehicle'         , 7       , True         , False        , (119, 11, 32) ),
    Label(  'license plate'        , -1 ,       -1 , 'vehicle'         , 7       , False        , True         , (  0,  0,142) ),
]

In [18]:
# Create color to trainid mapping
COLOR_TO_TRAINID = {label.color: label.trainId for label in labels if label.trainId != 255}

In [19]:
def find_closest_labels(mask, mapping):
    
    closest_distance = np.full([mask.shape[0], mask.shape[1]], np.inf) # Large initial distance
    closest_category = np.full([mask.shape[0], mask.shape[1]], 255, dtype=np.int32) # Initialize to unlabeld category 255  

    # Iterate over every color mapping
    for color, train_id in mapping.items():
        # Ensure color is a NumPy array
        color = np.array(color, dtype=np.int32)

        # Compute Euclidean distance between the mask and the current color in the mapping
        dist = np.linalg.norm(mask - color.reshape(1, 1, 3), axis=-1)

        # Find where the current color is the closest so far
        is_closer = dist < closest_distance

        # Only update where `is_closer` is True
        closest_distance[is_closer] = dist[is_closer]
        closest_category[is_closer] = train_id  # Assign train_id value

    return closest_category

In [20]:
# Create custom dataset class
class CityscapesDataset(Dataset):
    def __init__(self, root_dir, split='train', transform=None, target_transform=None):
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.target_transform = target_transform

        # Paths for images and labels
        self.img_dir = os.path.join(root_dir, split, "img")
        self.mask_dir = os.path.join(root_dir, split, "label")

        # List of image files
        self.img_filenames = sorted(os.listdir(self.img_dir))
        self.mask_filenames = sorted(os.listdir(self.mask_dir))

    def __len__(self):
        return len(self.img_filenames)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_filenames[idx])
        mask_path = os.path.join(self.mask_dir, self.mask_filenames[idx])

        # Load image and label
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("RGB")

        # Convert mask to class labels using the color mapping
        mask = find_closest_labels(np.array(mask), COLOR_TO_TRAINID)
        # Replace any unknown pixels with 255
        mask[mask == -1] = 255
        
        mask = torch.tensor(mask, dtype=torch.long).unsqueeze(0)

        # Apply transforms
        if self.transform:
            image = self.transform(image)
        else:
            image = transforms.ToTensor()(image) # Ensure image is a tensor
            
        if self.target_transform:
            mask = self.target_transform(mask)

        return image, mask

In [21]:
# Define the dice loss metric
def dice_loss(pred, target, epsilon=1e-6):
    # Apply softmax to get probabilities for each class from the logits
    pred = F.softmax(pred, dim=1)

    # One-hot encode the target
    target_one_hot = F.one_hot(target.long(), num_classes=pred.shape[1])
    target_one_hot = target_one_hot.permute(0, 3, 1, 2).float()

    # Compute the intersection and union
    intersection = torch.sum(pred_flat * target_flat, dim=2)
    union = torch.sum(pred_flat, dim=2) + torch.sum(target_flat, dim=2)

    # Dice coefficient and loss
    dice = (2.0 * intersection + epsilon) / (union + epsilon)
    return 1 - dice.mean()

In [22]:
# Define transformations. Since the images are pretty small (256x96), random cropping would remove
# too much valuable information. If blurring the image, a smaller kernel should be applied. Color jitter
# should be applied conservatively to avoid excessive distortion. Scaling also should be limited since 
# the images are small.
train_image_transform = transforms.Compose([
    transforms.Resize((256, 96)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),  # Color jitter
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_label_transform = transforms.Compose([
    transforms.Resize((256, 96), interpolation=transforms.InterpolationMode.NEAREST)  
])

val_image_transform = transforms.Compose([
    transforms.Resize((256, 96)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_label_transform = transforms.Compose([
    transforms.Resize((256, 96), interpolation=transforms.InterpolationMode.NEAREST)
])

In [23]:
# Create CityscapesDataset objects for the training and validation datasets
train_dataset = CityscapesDataset(
    root_dir=base_dir,
    split="train",
    transform=train_image_transform,
    target_transform=train_label_transform)

val_dataset = CityscapesDataset(
    root_dir=base_dir,
    split="val",
    transform=val_image_transform,
    target_transform=val_label_transform)

In [24]:
# Load the DeepLabV3 model with the ResNet50 backbone
model = deeplabv3_resnet50(weights=DeepLabV3_ResNet50_Weights.DEFAULT)

In [25]:
# Modify the model's final layer for the Cityscapes dataset
num_classes = len(labels)
model.classifier[4] = torch.nn.Conv2d(256, num_classes, kernel_size=1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/home/abey/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [26]:
# Define learning rate scheduler with warmup
def get_linear_warmup_scheduler(optimizer, base_lr, num_epochs, num_warmup=3):
    def lr_lambda(epoch):
        return min(1.0, (epoch+1) / num_warmup)

    return lr_scheduler.LambdaLR(optimizer, lr_lambda)

In [27]:
# Define hyperparameters
criterion = nn.CrossEntropyLoss(ignore_index=255) # pixels with 255 not included in loss
batch_size = 32
learning_rate = 0.001
decay = 0.001
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=decay)
num_epochs = 10
scheduler = get_linear_warmup_scheduler(optimizer, learning_rate, num_epochs)

best_model_path = "best_model.pth" # Path to save the best model based on validation loss

In [28]:
# Create dataloaders for the training and validation datasets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [31]:
def train_model(model, train_loader, val_loader, num_epochs, optimizer, scheduler):
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        print(f"Epoch {epoch+1}: LR = {scheduler.get_last_lr()[0]:.6f}")

        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)

            optimizer.zero_grad()
            outputs = model(images)['out'] # DeepLabV3 outputs a dict with 'out'
            
            loss = criterion(outputs, masks.squeeze(1).long())
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss/len(train_loader):.4f}")

        # Switch to eval for validation
        model.eval()
        
        with torch.no_grad():
            val_loss = 0

            for images, masks in val_loader:
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)['out']
                loss = criterion(outputs, masks.squeeze(1).long())
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        print(f"Validation Loss: {avg_val_loss:.4f}")

        # Save the model if the validation loss has improved
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), best_model_path)
            print(f"New best model saved with validation loss: {best_val_loss:.4f}")

        # Step the learning rate scheduler
        scheduler.step()

In [32]:
train_model(model, train_loader, val_loader, num_epochs, optimizer, scheduler)

Epoch 1: LR = 0.000333
Epoch [1/10], Train Loss: 1.1057
Validation Loss: 1.0012
New best model saved with validation loss: 1.0012
Epoch 2: LR = 0.000667
Epoch [2/10], Train Loss: 1.0808
Validation Loss: 1.0774
Epoch 3: LR = 0.001000
Epoch [3/10], Train Loss: 1.0460
Validation Loss: 1.0414
Epoch 4: LR = 0.001000
Epoch [4/10], Train Loss: 0.9794
Validation Loss: 1.0751
Epoch 5: LR = 0.001000
Epoch [5/10], Train Loss: 0.9239
Validation Loss: 1.3617
Epoch 6: LR = 0.001000
Epoch [6/10], Train Loss: 0.8884
Validation Loss: 1.0338
Epoch 7: LR = 0.001000
Epoch [7/10], Train Loss: 0.8675
Validation Loss: 0.8378
New best model saved with validation loss: 0.8378
Epoch 8: LR = 0.001000
Epoch [8/10], Train Loss: 0.8330
Validation Loss: 1.0182
Epoch 9: LR = 0.001000
Epoch [9/10], Train Loss: 0.8189
Validation Loss: 1.2605
Epoch 10: LR = 0.001000
Epoch [10/10], Train Loss: 0.8037
Validation Loss: 0.8466


In [125]:
for batch in train_loader:
    image = batch[0]
    mask = batch[1]

    print(f"Image shape: {image.shape}")  # Should be (3, H, W)
    print(f"Mask shape: {mask.shape}")    # Should be (1, H, W)
    break

Image shape: torch.Size([32, 3, 256, 96])
Mask shape: torch.Size([32, 1, 256, 96])
